import or install cloudant database library

In [1]:
import sys
try:
    import cloudant
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install --pre cloudant 
    else:
        !pip install --user --pre cloudant
    import cloudant

    100% |████████████████████████████████| 71kB 2.2MB/s ta 0:00:01


Credentials for cloudant database on Bluemix

In [2]:
# The code was removed by DSX for sharing.

Define the indexes that will be put into Cloudant

In [24]:

indexes = [
    { 
        'name': 'create standard geo index',
        'indexName' : 'geoidx',
        'javascript' : '\
function(doc) { \n\
    if (doc.geometry && doc.geometry.coordinates && doc.geometry.coordinates.length > 0) { \n\
        st_index(doc.geometry);\n\
    } \n\
} \
        ',
    },
    {
        'name': 'create walking index',
        'indexName': 'walkidx',
        'javascript' : '\
function (doc) { \n\
    // anything below a certain speed (got to find units) is someone walking \n\
    if (doc.properties.gpsSpeed > 2.0) \n\
        return; \n\
    // make sure the document has a geometry for gepspatial handling \n\
    if (doc.geometry && doc.geometry.coordinates && doc.geometry.coordinates.length > 0) { \n\
        st_index(doc.geometry);\n\
    }\n\
} \
        ',
    },
    {
        'name': 'create car index',
        'indexName': 'caridx',
        'javascript' : '\
function (doc) { \n\
    // anything above a certain speed is someone in a car \n\
    if (doc.properties.gpsSpeed <= 2.0) \n\
        return; \n\
    // make sure the document has a geometry for gepspatial handling \n\
    if (doc.geometry && doc.geometry.coordinates && doc.geometry.coordinates.length > 0) { \n\
        st_index(doc.geometry);\n\
    }\n\
} \
        ',
    },
    {
        'name': 'create -85 dbm index - strong signal',
        'indexName': '85idx',
        'javascript' : '\
function (doc) { \n\
    // ignore anything below signal strength -85dbm \n\
    if (doc.properties.signalStrength < -85) \n\
        return; \n\
    // make sure the document has a geometry for gepspatial handling \n\
    if (doc.geometry && doc.geometry.coordinates && doc.geometry.coordinates.length > 0) { \n\
        st_index(doc.geometry);\n\
    }\n\
}\
        ',
    },
        {
        'name': 'create -100 dbm index - weak signal',
        'indexName': '100idx',
        'javascript' : '\
function (doc) { \n\
    // ignore anything above signal strength -100dbm \n\
    if (doc.properties.signalStrength > -100) \n\
        return; \n\
    // make sure the document has a geometry for gepspatial handling \n\
    if (doc.geometry && doc.geometry.coordinates && doc.geometry.coordinates.length > 0) { \n\
        st_index(doc.geometry);\n\
    }\n\
}\
        ',
    },

]

create the indexes into cloudant

In [26]:
from cloudant import design_document
from cloudant import cloudant_bluemix

with cloudant_bluemix(
    {
        "cloudantNoSQLDB": [
            {
                "credentials": credentials_1,
                "syslog_drain_url": '/dev/null',
                "label": "cloudantNoSQLDB",
                "provider": 'IBM',
                "plan": "Lite",
                "name": "Cloudant NoSQL DB"
            }
        ]
    },
    'Cloudant NoSQL DB') as client:
        db = client['rf-checker-geojson']
        ddoc = design_document.DesignDocument(db, '_design/geosignal')

        # insert Indexes
        stIndexes = {x['indexName']: {'index' : x['javascript']} for x in indexes }
        # Add the Cloudant Geo index to ``st_indexes`` and save the design document
        ddoc['st_indexes'] = stIndexes
        ddoc.save()

Add the geohash for each GeoJson document in the database.
This will allow us to group signals together into a group
We will add two geohashes that will be eventually indexed/or added to views
The precision of each geohash is approximate depending on the digits of precision. At the equator the shap is more a square, but as we move towards either pole it becomes more a rectangle
+ 7 digit precision will be approx a 76 meter (249 feet) square at the equator. 
+ 8 digit precision will be approx 19 meter (62 feet) square at the equator. 
+ 9 digit precision will be approx 2.4 meter (7.8 feet) square at the equator. 

The information was cleamed from an article by [Stefan Kruger](https://medium.com/@xpqz/geohashing-for-fun-and-profit-in-cloudant-989420fe1e73) and from a [StackExchange posting](https://gis.stackexchange.com/questions/115280/what-is-the-precision-of-a-geohash)

In [39]:
try:
    import Geohash
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install Geohash
    else:
        !pip install --user Geohash
    import Geohash

def addGeoHash(doc):
    lat = doc['geometry']['coordinates'][1]
    long = doc['geometry']['coordinates'][0]
    doc['geohash'] = [
        Geohash.encode(lat, long, precision=7),
        Geohash.encode(lat, long, precision=8),
        Geohash.encode(lat, long, precision=9)
    ]

In [41]:
from cloudant import cloudant_bluemix
import math
import time

lastDoc = {}
with cloudant_bluemix(
    {
        "cloudantNoSQLDB": [
            {
                "credentials": credentials_1,
                "syslog_drain_url": '/dev/null',
                "label": "cloudantNoSQLDB",
                "provider": 'IBM',
                "plan": "Lite",
                "name": "Cloudant NoSQL DB"
            }
        ]
    },
    'Cloudant NoSQL DB') as client:
        db = client['rf-checker-geojson']
        # Run a query
        query = 'id:signal-strength'
        limit = 200
        queryResult = db.get_search_result('geosignal', 'ididx',
                                query=query,
                                include_docs=True,
                                limit=limit)
        rangeHigh = math.ceil(queryResult['total_rows'] / limit)
        for i in range(0, rangeHigh):
            bookmark = queryResult['bookmark']
            for doc in queryResult['rows']:
                # add the geohash to the documents and then update them in the database     
                if 'doc' in doc:
                    doc = doc['doc']
                addGeoHash(doc)
                lastDoc = doc
            # Now bulk update the database
            # remove the annoying 'doc' attribute
            db.bulk_docs([item['doc'] for item in queryResult['rows']])
            # iterate for next set of documents
            queryResult = db.get_search_result('geosignal', 'ididx', 
                                                query=query,
                                                include_docs=True,
                                                bookmark=bookmark,
                                                limit=limit)
            # The free version of cloudant limits the number of call/sec to 5
            if (i + 1) % 5 == 0:
                time.sleep(0.5)
 # print out the last doc  
lastDoc

TypeError: range() integer end argument expected, got float.

In [44]:
from cloudant import cloudant_bluemix
import math
import time

lastDoc = {}
with cloudant_bluemix(
    {
        "cloudantNoSQLDB": [
            {
                "credentials": credentials_1,
                "syslog_drain_url": '/dev/null',
                "label": "cloudantNoSQLDB",
                "provider": 'IBM',
                "plan": "Lite",
                "name": "Cloudant NoSQL DB"
            }
        ]
    },
    'Cloudant NoSQL DB') as client:
        db = client['rf-checker-geojson']
        # Run a query
        query = 'id:signal-strength'
        limit = 200
        queryResult = db.get_search_result('geosignal', 'ididx',
                                query=query,
                                include_docs=True,
                                limit=limit)
        lastDoc = round(queryResult['total_rows'] / limit) 
        # print out the last doc  
lastDoc

7.0